In [2]:
import pathlib

architecture_result_paths = list(pathlib.Path("../results/gorillas").glob("*"))
print("Number of architectures:", len(list(architecture_result_paths)))

Number of architectures: 14


In [3]:
import json
import pandas as pd
import statistics

tials_that_should_exist = []
for architecture_result_path in architecture_result_paths:
    tials_that_should_exist.append(architecture_result_path.name)
    tials_that_should_exist.append(architecture_result_path.name + "-QAT")
    tials_that_should_exist.append(architecture_result_path.name + "-PTQ")
    tials_that_should_exist.append(architecture_result_path.name + "-QAT&KD")
    

dataframe_list = []
trials_that_exist = []
for architecture_result_path in architecture_result_paths:
    trials = list(architecture_result_path.glob("*"))
    
    for trial in trials:
        config_dict = json.loads((trial / "config.json").read_text())
        metrics_dict = json.loads((trial / "metrics.json").read_text())
        
        name = trial.parent.name
        if "use_quantization_aware_training" not in config_dict.keys():
            continue
        
        if config_dict["use_quantization_aware_training"]["0"]:
            name += "-QAT"
        if config_dict["loss_mode"]["0"] == "distillation/offline/response-based":
            name += "&KD"
            
        if "aggregated/cxlkfold/val/embeddings/knn/accuracy_max" not in metrics_dict.keys():
            continue
        else:
            
            important_metrics = [
                'val/embeddings/knn/accuracy_max', "val/embeddings/knn/precision_max", "val/embeddings/knn/f1_max", "val/embeddings/knn5/accuracy_max", "val/embeddings/knn5/precision_max", "val/embeddings/knn5/f1_max", "val/embeddings/knn5/accuracy_top5_max",
                'val/embeddings/knn_crossvideo/accuracy_max', "val/embeddings/knn_crossvideo/precision_max", "val/embeddings/knn_crossvideo/f1_max", "val/embeddings/knn5_crossvideo/accuracy_max", "val/embeddings/knn5_crossvideo/precision_max", "val/embeddings/knn5_crossvideo/f1_max", "val/embeddings/knn5_crossvideo/accuracy_top5_max",
            ]
            dataframe_entry = {"name": name}
            for metric in important_metrics:
                values = []
                metric_name = metric.replace("knn/","knn-1/").replace('knn_', "knn-1_").split("/")[-2].replace("knn5", "knn-5")+ " " + metric.split("/")[-1].replace("_max", "")
                for fold in range(5):
                    values.append(metrics_dict[f"cxlkfold/fold-{fold}/" + metric]['0'])
                
                    
                dataframe_entry[metric_name + " stdev"] = statistics.stdev(values)
                dataframe_entry[metric_name + " mean"] = statistics.mean(values)
                dataframe_entry[metric_name + " min"] = min(values)
                dataframe_entry[metric_name + " max"] = max(values)
                dataframe_entry[metric_name] = metrics_dict["aggregated/cxlkfold/" + metric]["0"]
                    
            dataframe_list.append(dataframe_entry)
            trials_that_exist.append(name)
            
dataframe = pd.DataFrame(dataframe_list)
dataframe.to_csv("gorillas.csv", index=False)
    
        

In [4]:
# get data from ptq

import pathlib
import pandas as pd
import json
import statistics

architecture_result_paths = list(pathlib.Path("../runs").glob("*"))
print("Number of architectures:", len(list(architecture_result_paths)))

for architecture_path in architecture_result_paths:
    results = json.loads((architecture_path / "results.json").read_text())
    
    dataframe_entry = {"name": architecture_path.name + "-PTQ"}
    for metric in important_metrics:
        metric_name = metric.replace("knn/","knn-1/").replace('knn_', "knn-1_").split("/")[-2].replace("knn5", "knn-5")+ " " + metric.split("/")[-1].replace("_max", "")
        knn = metric.replace("knn/", "knn1/").replace('knn_', "knn1_").split("/")[-2]
        metric_name_2 = metric.split("/")[-1].replace("_max", "")
        
        values = []
        for fold in range(5):
            try:
                values.append(results[f"quantized-fold-{fold}"][knn][metric_name_2] )
            except Exception as e:
                print(f"Error with {architecture_path.name} {knn} {metric_name_2}")
                values.append(0)
            
        dataframe_entry[metric_name + " stdev"] = statistics.stdev(values)
        dataframe_entry[metric_name + " mean"] = statistics.mean(values)
        dataframe_entry[metric_name + " min"] = min(values)
        dataframe_entry[metric_name + " max"] = max(values)
        dataframe_entry[metric_name] = min(values)
        
            
    dataframe_list.append(dataframe_entry)
    trials_that_exist.append(architecture_path.name + "-PTQ")
    
dataframe = pd.DataFrame(dataframe_list)
dataframe.to_csv("gorillas.csv", index=False)

Number of architectures: 14


In [5]:
from math import sqrt
import matplotlib 

SPINE_COLOR = 'gray'

def latexify(fig_width=None, fig_height=None, columns=1):
    """Set up matplotlib's RC params for LaTeX plotting.
    Call this before plotting a figure.

    Parameters
    ----------
    fig_width : float, optional, inches
    fig_height : float,  optional, inches
    columns : {1, 2}
    """

    # code adapted from http://www.scipy.org/Cookbook/Matplotlib/LaTeX_Examples

    # Width and max height in inches for IEEE journals taken from
    # computer.org/cms/Computer.org/Journal%20templates/transactions_art_guide.pdf

    assert(columns in [1,2])

    if fig_width is None:
        fig_width = 3.39 if columns==1 else 6.9 # width in inches
        #fig_width = 12 if columns==1 else 17 # width in inches

    if fig_height is None:
        golden_mean = (sqrt(5)-1.0)/2.0    # Aesthetic ratio
        fig_height = fig_width*golden_mean # height in inches

    MAX_HEIGHT_INCHES = 8.0
    if fig_height > MAX_HEIGHT_INCHES:
        print("WARNING: fig_height too large:" + fig_height + 
              "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
        fig_height = MAX_HEIGHT_INCHES

    params = {'backend': 'ps',
              'text.latex.preamble': '\\usepackage{gensymb}',
              'axes.labelsize': 8, # fontsize for x and y labels (was 10)
              'axes.titlesize': 8,
              'lines.linewidth': 0.5,
              'axes.linewidth': 0.5,
              #'text.fontsize': 8, # was 10
              'legend.fontsize': 8, # was 10
              'xtick.labelsize': 8,
              'ytick.labelsize': 8,
              'lines.markersize': 2,
              'text.usetex': True,
              'figure.figsize': [fig_width,fig_height],
              'font.family': 'serif'
    }

    matplotlib.rcParams.update(params)


def format_axes(ax):

    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)

    for spine in ['left', 'bottom']:
        ax.spines[spine].set_color(SPINE_COLOR)
        ax.spines[spine].set_linewidth(0.5)

    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_tick_params(direction='out', color=SPINE_COLOR)

    return ax

latexify()



In [6]:
tails_that_are_missing = set(tials_that_should_exist) - set(trials_that_exist)
tails_that_are_missing = sorted(list(tails_that_are_missing))

missing_kd = []
missing_baseline = []
missing_qat = []
missing_ptq = []
for tail in tails_that_are_missing:
    if "&KD" in tail:
        missing_kd.append(tail)
    elif "-QAT" in tail:
        missing_qat.append(tail)
    elif "-PTQ" in tail:
        missing_ptq.append(tail)
    else:
        missing_baseline.append(tail)
        
with open("missing.txt", "w") as file:
    file.write("Missing KD:\n")
    for tail in missing_kd:
        file.write(tail + "\n")
        
    file.write("\nMissing QAT:\n")
    for tail in missing_qat:
        file.write(tail + "\n")
        
    file.write("\nMissing Baseline:\n")
    for tail in missing_baseline:
        file.write(tail + "\n") 
        
    file.write("\nMissing PTQ:\n")
    for tail in missing_ptq:
        file.write(tail + "\n")

In [7]:
import pandas as pd
import re

def generate_latex_table(df, metric_names):
    # Define the model size order
    model_size_order = ['Small', 'Medium', 'Base', 'Large', '18', '50', '152']

    # Extract architecture and model size for sorting
    def extract_arch_and_size(name):
        # Match the last occurrence of the model size (assumed to be at the end)
        match = re.search(r'(Small|Medium|Base|Large|18|50|152)$', name)
        if match:
            size = match.group(0)
            arch = name[:match.start()].strip()
        else:
            size = ''
            arch = name.strip()
        return arch, size
    
    # Apply extraction to create separate columns
    df[['Arch_Base', 'Model_Size']] = df['name'].apply(lambda x: pd.Series(extract_arch_and_size(x)))

    # Sort by architecture and then by the model size order
    df['Size_Rank'] = df['Model_Size'].apply(lambda x: model_size_order.index(x) if x in model_size_order else len(model_size_order))
    df = df.sort_values(by=['Arch_Base', 'Size_Rank'])
    
    # If an architecture has the same name but multiple rows choose the row wit the highest knn-1 accuracy
    df = df.sort_values(by=['knn-1 accuracy'], ascending=False).drop_duplicates(subset=['Arch_Base'], keep='first')

    # Group the data by architecture and calculate metrics
    results = []
    for arch_base, group in df.groupby('Arch_Base'):
        for _, sub_group in group.iterrows():
            arch = f"{sub_group['Arch_Base']} {sub_group['Model_Size']}".strip()
            row = [arch.replace("Embedding-", "").replace("-CXL-OpenSet", "").replace("-", " ").replace("&", "\&")]
            for metric in metric_names.keys():
                value = sub_group[metric]  # Get value for each metric
                row.append(f'{value:.4f}')  # Format to four decimal places
            results.append(row)

    # Build LaTeX table
    latex_table = r'''\begin{table}[H]
    \centering
    \begin{tabular}{l c c c c c c}
    \toprule
    & \multicolumn{3}{c}{KNN-1} & \multicolumn{3}{c}{KNN-5} \\ 
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    Model  & ''' + ' & '.join(metric_names.values()) + r''' \\ 
    \midrule
'''

    for result in results:
        latex_table += '    ' + ' & '.join(result) + r' \\' + '\n'

    latex_table += r'''    \bottomrule
    \end{tabular}
    \caption{Metrics for each architecture.}
    \label{tab:metrics-architecture}
\end{table}'''

    return latex_table

# Example usage
# Define the metric names you want to use
metric_names = {
    'knn-1 accuracy': 'Accuracy',
    'knn-1 precision': 'Precision',
    'knn-1 f1': 'F1 Score',
    'knn-5 accuracy': 'Accuracy',
    'knn-5 precision': 'Precision',
    'knn-5 f1': 'F1 Score'
}

# df = your_dataframe_here
latex_code = generate_latex_table(dataframe, metric_names)
print(latex_code)


\begin{table}[H]
    \centering
    \begin{tabular}{l c c c c c c}
    \toprule
    & \multicolumn{3}{c}{KNN-1} & \multicolumn{3}{c}{KNN-5} \\ 
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    Model  & Accuracy & Precision & F1 Score & Accuracy & Precision & F1 Score \\ 
    \midrule
    EfficientNet Large & 0.7723 & 0.7558 & 0.7516 & 0.6964 & 0.6545 & 0.6585 \\
    EfficientNet Large PTQ & 0.6429 & 0.5888 & 0.6098 & 0.5759 & 0.5449 & 0.5309 \\
    EfficientNet Large QAT & 0.4277 & 0.4078 & 0.4153 & 0.4116 & 0.3660 & 0.3822 \\
    EfficientNet Large QAT\&KD & 0.7143 & 0.7144 & 0.7136 & 0.2598 & 0.1441 & 0.1663 \\
    EfficientNet Medium & 0.8404 & 0.8274 & 0.8234 & 0.7589 & 0.7157 & 0.7212 \\
    EfficientNet Medium PTQ & 0.7946 & 0.7728 & 0.7606 & 0.7232 & 0.6700 & 0.6604 \\
    EfficientNet Medium QAT & 0.6741 & 0.6624 & 0.6620 & 0.6062 & 0.5795 & 0.5791 \\
    EfficientNet Medium QAT\&KD & 0.6232 & 0.6066 & 0.6095 & 0.5652 & 0.5432 & 0.5426 \\
    EfficientNet Small & 0.8203 & 0.8058 &

In [11]:
import pandas as pd
import re

# Mapping of architectures to pretraining datasets
architecture_pretrained_mapping = {
    "efficientnet": "Imagenet 21k",
    "resnet": "Imagenet 1k",
    "swin": "Imagenet 22k\&1k",
    "vit": "Imagenet 21k",
    "dino": "DINO Dataset",
    "clip": "CLIP Dataset",
}

# Function to extract architecture and variation type, removing "Embedding-" prefix
def extract_architecture_and_type(name):
    base_name = re.sub(r'^Embedding-', '', name)
    base_name = re.sub(r'-QAT&KD|-QAT|-PTQ|-CXL-OpenSet|-CXL-Open', '', base_name)
    
    if 'PTQ' in name:
        var_type = 'PTQ'
    elif 'QAT&KD' in name:
        var_type = 'QAT\&KD'
    elif 'QAT' in name:
        var_type = 'QAT'
    else:
        var_type = 'Baseline'
    
    base_name_new = ""
    for architecture, pretrained in architecture_pretrained_mapping.items():
        if architecture in base_name.lower():
            base_name_new = base_name + "\n" + pretrained
            
    return base_name_new, var_type

# Function to generate LaTeX table with highlighted max values
def generate_latex_table_with_configs(df, metric):
    # Extract architecture and variation
    df['Architecture'], df['Variation'] = zip(*df['name'].apply(extract_architecture_and_type))

    # Order by model size
    model_size_order = ['Small', 'Medium', 'Base', 'Large', '18', '50', '152']
    arch_order = sorted(
        df['Architecture'].unique(), 
        key=lambda x: (
            re.sub(r'(Small|Medium|Base|Large|18|50|152)', '', x), 
            model_size_order.index(re.search(r'(Small|Medium|Base|Large|18|50|152)', x).group(0)) if re.search(r'(Small|Medium|Base|Large|18|50|152)', x) else -1
        )
    )

    # Group by architecture and variation, taking max accuracy
    grouped_df = df.groupby(['Architecture', 'Variation'])[metric].max().unstack()

    # Reorder based on architecture order
    grouped_df = grouped_df.loc[arch_order]

    # Build LaTeX table
    latex_table = r'''\begin{table}[H]
    \centering
    \begin{tabular}{l c c c c}
    \toprule
    Model & Baseline & PTQ & QAT & QAT\&KD \\
    \midrule
'''

    for arch, row in grouped_df.iterrows():
        formatted_row = [arch.replace("\n", " ")]
        for config in ['Baseline', 'PTQ', 'QAT', 'QAT\&KD']:
            value = row.get(config, 'N/A')
            if pd.isna(value):
                formatted_row.append('N/A')
            else:
                # Bold the maximum value in the row
                if value == row.max():
                    formatted_row.append(f'\\textbf{{{value:.4f}}}')
                else:
                    formatted_row.append(f'{value:.4f}')
        latex_table += '    ' + ' & '.join(formatted_row) + r' \\' + '\n'

    latex_table += r'''    \bottomrule
    \end{tabular}
    \caption{KNN-1 Accuracy across different configurations.}
    \label{tab:knn1-accuracy-configs}
\end{table}'''

    return latex_table

# Example usage with the DataFrame `dataframe`
latex_code = generate_latex_table_with_configs(dataframe, 'knn-1_crossvideo accuracy')
print(latex_code)


\begin{table}[H]
    \centering
    \begin{tabular}{l c c c c}
    \toprule
    Model & Baseline & PTQ & QAT & QAT\&KD \\
    \midrule
    EfficientNet-Small Imagenet 21k & 0.3520 & 0.3226 & 0.2853 & \textbf{0.4191} \\
    EfficientNet-Medium Imagenet 21k & \textbf{0.3752} & 0.3474 & 0.2194 & 0.3096 \\
    EfficientNet-Large Imagenet 21k & 0.3170 & 0.1158 & 0.1895 & \textbf{0.6413} \\
    EfficientNetRWM Imagenet 21k & 0.4434 & 0.4632 & 0.4212 & \textbf{0.6781} \\
    ResNet-18 Imagenet 1k & 0.2562 & 0.1789 & 0.2607 & \textbf{0.3229} \\
    ResNet-50 Imagenet 1k & 0.2787 & 0.1299 & 0.2673 & \textbf{0.4151} \\
    ResNet-152 Imagenet 1k & 0.2302 & 0.1895 & 0.2627 & \textbf{0.5878} \\
    SwinV2-Small Imagenet 22k\&1k & 0.4423 & 0.5053 & 0.4255 & \textbf{0.7392} \\
    SwinV2-Base Imagenet 22k\&1k & 0.5782 & 0.5368 & 0.5275 & \textbf{0.6854} \\
    SwinV2-Large Imagenet 22k\&1k & 0.5549 & 0.4737 & 0.5279 & \textbf{0.6906} \\
    ViT-Base Imagenet 21k & 0.3194 & 0.0737 & 0.4016 & \textbf{

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from matplotlib.container import ErrorbarContainer

architecture_pretrained_mapping = {
    "efficientnet": "Imagenet 21k",
    "resnet": "Imagenet 1k",
    "swin": "Imagenet 22k\&1k",
    "vit": "Imagenet 21k",
    "dino": "DINO Dataset",
    "clip": "CLIP Dataset",
}

# Function to extract architecture and variation type, removing "Embedding-" prefix
def extract_architecture_and_type(name):
    base_name = re.sub(r'^Embedding-', '', name)
    base_name = re.sub(r'-QAT&KD|-QAT|-PTQ|-CXL-OpenSet|-CXL-Open', '', base_name)
    
    if 'PTQ' in name:
        var_type = 'PTQ'
    elif 'QAT&KD' in name:
        var_type = 'QAT\&KD'
    elif 'QAT' in name:
        var_type = 'QAT'
    else:
        var_type = 'Baseline'
    
    base_name_new = ""
    for architecture, pretrained in architecture_pretrained_mapping.items():
        if architecture in base_name.lower():
            base_name_new = base_name + "\n" + pretrained
            
    return base_name_new, var_type

# Function to plot bar charts with error bars for a given metric
def plot_metric_bars_with_error(df, metric, title):
    # Extract architecture and variation
    df['Architecture'], df['Variation'] = zip(*df['name'].apply(extract_architecture_and_type))

    # Order by model size
    model_size_order = ['Small', 'Medium', 'Base', 'Large', '18', '50', '152']
    arch_order = sorted(
        df['Architecture'].unique(), 
        key=lambda x: (
            re.sub(r'(Small|Medium|Base|Large|18|50|152)', '', x), 
            model_size_order.index(re.search(r'(Small|Medium|Base|Large|18|50|152)', x).group(0)) if re.search(r'(Small|Medium|Base|Large|18|50|152)', x) else -1
        )
    )

    # Group by architecture and variation, taking max accuracy and standard deviation
    grouped_df = df.groupby(['Architecture', 'Variation'])[metric].max().unstack()
    grouped_df_stdev = df.groupby(['Architecture', 'Variation'])[f'{metric} stdev'].max().unstack()

    # Reorder based on architecture order
    grouped_df = grouped_df.loc[arch_order]
    grouped_df_stdev = grouped_df_stdev.loc[arch_order]

    # Plot the bar chart with error bars
    fig, ax = plt.subplots(figsize=(14, 11))
    grouped_df[['Baseline', 'PTQ', 'QAT', 'QAT\&KD']].plot(kind='bar', ax=ax, yerr=grouped_df_stdev[['Baseline', 'PTQ', 'QAT', 'QAT\&KD']], capsize=2)

    # Set title and labels
    plt.title(title.replace("&", "\&"))
    plt.ylabel(metric.replace("_", " ").title())
    plt.xlabel('Architecture')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Variation')
    plt.tight_layout()
    plt.show()
    plt.savefig(title + ".pdf")


plot_metric_bars_with_error(dataframe, 'knn-1 accuracy', 'KNN-1 Accuracy for Each Architecture (Baseline, PTQ, QAT, and QAT KD)')
plot_metric_bars_with_error(dataframe, 'knn-1_crossvideo accuracy', 'KNN-1 Crossvideo Accuracy for Each Architecture (Baseline, PTQ, QAT, and QAT KD)')
plot_metric_bars_with_error(dataframe, 'knn-1_crossvideo precision', 'KNN-1 Crossvideo Precision for Each Architecture (Baseline, PTQ, QAT, and QAT KD)')

In [10]:
groups = {
    "CNNs": ["resnet", "efficientnet"],
    "Transformers": ["vit", "swin"],
    "EfficientNet": ["efficientnet"],
    "ResNet": ["resnet"],
    "Swin Transformer": ["swin"],
    "ViT": ["vit"],
    "All": ["resnet", "efficientnet", "vit", "swin"],
}

df = dataframe

for group_name, group in groups.items():
    filtered_df = df[df['Architecture'].isin(group)]